In this notebook, I attempt to address [ER team issue #72](https://github.com/hackoregon/emergency-response/issues/72), 'Get incident rates for each FMA'

4/12
So far, I have calculated FMA incident rates by square mile (also split by medical vs fire/explosion). If the rollups look acceptable to the team, I can append them to `fma_api_rollup` in the db and then can add incidents per capita and household. 

4/27
Updated queries to accommodate the fact that fma_shapes.fma datatype is now INTEGER instead of VARCHAR. CAST  fma_shapes.fma as text with leading zero if less than 10 to enable JOIN to fma_shapes.fma which is VARCHAR. Added some columns for csv export. 

In [1]:
import os 
from dotenv import load_dotenv, find_dotenv
import psycopg2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys

%matplotlib inline

In [2]:
# walk root diretory to find and load .env file w/ AWS host, username and password
load_dotenv(find_dotenv())

True

In [3]:
# connect to postgres
def pgconnect():
    try:
        conn = psycopg2.connect(database=os.environ.get("erdatabase"), user=os.environ.get("eruser"), 
                            password = os.environ.get("erpassword"), 
                            host=os.environ.get("erhost"), port=os.environ.get("erport"))
        print("Opened database successfully")
        return conn
    
    except psycopg2.Error as e:
        print("I am unable to connect to the database")
        print(e)
        print(e.pgcode)
        print(e.pgerror)
        print(traceback.format_exc())
        return None

In [4]:
def pquery(QUERY):
    '''
    takes SQL query string, opens a cursor, executes query in psql, and pulls results into pandas df
    '''
    conn = pgconnect()
    cur = conn.cursor()
    
    try:
        print("SQL QUERY = "+QUERY)
        cur.execute("SET statement_timeout = 0")
        cur.execute(QUERY)
        # Extract the column names and insert them in header
        col_names = []
        for elt in cur.description:
            col_names.append(elt[0])    
    
        D = cur.fetchall() #convert query result to list
        # Create the dataframe, passing in the list of col_names extracted from the description
        return pd.DataFrame(D, columns=col_names)
        
        
    except Exception as e:
        print(e.pgerror)
            
    finally:
        conn.close()



In [5]:
# table of fma, fma_area, and number of incidents
QUERY1='''select s.fma, ST_Area(s.geom::geography) as fma_area,count(i.incident_id) as num_incidents
 FROM fma_shapes s
 INNER JOIN
 incident i
 ON
 LPAD(s.fma::text,2,'0') = i.fmarespcomp
 GROUP BY s.fma, fma_area
 ORDER BY num_incidents DESC;
'''

In [6]:
df1 = pquery(QUERY1)

Opened database successfully
SQL QUERY = select s.fma, ST_Area(s.geom::geography) as fma_area,count(i.incident_id) as num_incidents
 FROM fma_shapes s
 INNER JOIN
 incident i
 ON
 LPAD(s.fma::text,2,'0') = i.fmarespcomp
 GROUP BY s.fma, fma_area
 ORDER BY num_incidents DESC;



In [7]:
df1

,fma,fma_area,num_incidents
0,1,2.653218e+06,43139
1,7,1.477483e+07,42676
2,3,6.330879e+06,30723
3,11,1.374387e+07,30118
4,4,6.131893e+06,29829
5,13,8.638372e+06,29499
6,31,1.093237e+07,24451
7,25,1.295804e+07,22742
8,30,1.078235e+07,21593
9,19,1.084701e+07,21263


In [8]:
# same table but convert area to square miles (1m^2 = 3.861x10^-7 mile^2)
QUERY2='''select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
   LPAD(s.fma::text,2,'0') = i.fmarespcomp
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC
'''

In [9]:
df2 = pquery(QUERY2)

Opened database successfully
SQL QUERY = select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
   LPAD(s.fma::text,2,'0') = i.fmarespcomp
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC



In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
fma                   31 non-null int64
fma_area_mi           31 non-null float64
num_incidents         31 non-null int64
incidents_per_sqmi    31 non-null float64
dtypes: float64(2), int64(2)
memory usage: 1.0 KB


In [11]:
df2

,fma,fma_area_mi,num_incidents,incidents_per_sqmi
0,1,1.024407,43139,42111.178792
1,21,0.952173,12641,13275.945865
2,4,2.367524,29829,12599.238744
3,3,2.444352,30723,12568.974149
4,13,3.335275,29499,8844.547307
5,7,5.704560,42676,7481.032716
6,31,4.220988,24451,5792.719035
7,11,5.306509,30118,5675.671046
8,30,4.163065,21593,5186.804093
9,19,4.188032,21263,5077.086780


In [12]:
# look at just medical incidents
QUERY3='''select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
   LPAD(s.fma::text,2,'0') = i.fmarespcomp
  INNER JOIN incsitfound
    ON i.incsitfoundprm_id = incsitfound.incsitfound_id
  LEFT JOIN incsitfoundsub
    ON incsitfound.incsitfoundsub_id = incsitfoundsub.incsitfoundsub_id
  LEFT JOIN incsitfoundclass
    ON incsitfoundsub.incsitfoundclass_id = incsitfoundclass.incsitfoundclass_id 
 WHERE incsitfoundclass.incsitfoundclass_id = 3
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC;
'''


In [13]:
df3 = pquery(QUERY3)

Opened database successfully
SQL QUERY = select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
   LPAD(s.fma::text,2,'0') = i.fmarespcomp
  INNER JOIN incsitfound
    ON i.incsitfoundprm_id = incsitfound.incsitfound_id
  LEFT JOIN incsitfoundsub
    ON incsitfound.incsitfoundsub_id = incsitfoundsub.incsitfoundsub_id
  LEFT JOIN incsitfoundclass
    ON incsitfoundsub.incsitfoundclass_id = incsitfoundclass.incsitfoundclass_id 
 WHERE incsitfoundclass.incsitfoundclass_id = 3
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC;



In [14]:
df3

,fma,fma_area_mi,num_incidents,incidents_per_sqmi
0,1,1.024407,33984,33174.303996
1,21,0.952173,9233,9696.765143
2,4,2.367524,20167,8518.181895
3,3,2.444352,19821,8108.896807
4,13,3.335275,20171,6047.776661
5,7,5.704560,32937,5773.802010
6,11,5.306509,22245,4192.021463
7,30,4.163065,16836,4044.136235
8,19,4.188032,15518,3705.320635
9,9,3.451640,11703,3390.561845


In [15]:
# look at non-medical incidents
QUERY4='''select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
  LPAD(s.fma::text,2,'0') = i.fmarespcomp
  INNER JOIN incsitfound
    ON i.incsitfoundprm_id = incsitfound.incsitfound_id
  LEFT JOIN incsitfoundsub
    ON incsitfound.incsitfoundsub_id = incsitfoundsub.incsitfoundsub_id
  LEFT JOIN incsitfoundclass
    ON incsitfoundsub.incsitfoundclass_id = incsitfoundclass.incsitfoundclass_id 
 WHERE incsitfoundclass.incsitfoundclass_id != 3
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC;
'''

In [16]:
df4 = pquery(QUERY4)

Opened database successfully
SQL QUERY = select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
  LPAD(s.fma::text,2,'0') = i.fmarespcomp
  INNER JOIN incsitfound
    ON i.incsitfoundprm_id = incsitfound.incsitfound_id
  LEFT JOIN incsitfoundsub
    ON incsitfound.incsitfoundsub_id = incsitfoundsub.incsitfoundsub_id
  LEFT JOIN incsitfoundclass
    ON incsitfoundsub.incsitfoundclass_id = incsitfoundclass.incsitfoundclass_id 
 WHERE incsitfoundclass.incsitfoundclass_id != 3
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC;



In [17]:
df4

,fma,fma_area_mi,num_incidents,incidents_per_sqmi
0,1,1.024407,8762,8553.238336
1,3,2.444352,10695,4375.392329
2,4,2.367524,9124,3853.815223
3,21,0.952173,3299,3464.705752
4,13,3.335275,9127,2736.505755
5,7,5.704560,9353,1639.565540
6,11,5.306509,7575,1427.492137
7,19,4.188032,5637,1345.978375
8,9,3.451640,4574,1325.167041
9,25,5.003100,5978,1194.859293


In [18]:
# look at just fire/explosion incidents
QUERY5='''select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
LPAD(s.fma::text,2,'0') = i.fmarespcomp
  INNER JOIN incsitfound
    ON i.incsitfoundprm_id = incsitfound.incsitfound_id
  LEFT JOIN incsitfoundsub
    ON incsitfound.incsitfoundsub_id = incsitfoundsub.incsitfoundsub_id
  LEFT JOIN incsitfoundclass
    ON incsitfoundsub.incsitfoundclass_id = incsitfoundclass.incsitfoundclass_id 
 WHERE incsitfoundclass.incsitfoundclass_id IN (1,2)
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC;
'''

In [19]:
df5 = pquery(QUERY5)

Opened database successfully
SQL QUERY = select s.fma, ST_Area(s.geom::geography)*(.0000003861) as fma_area_mi, count(i.incident_id) as num_incidents,
count(i.incident_id)/(ST_Area(s.geom::geography)*(.0000003861)) as incidents_per_sqmi
 FROM fma_shapes s
 INNER JOIN
 incident i
   ON
LPAD(s.fma::text,2,'0') = i.fmarespcomp
  INNER JOIN incsitfound
    ON i.incsitfoundprm_id = incsitfound.incsitfound_id
  LEFT JOIN incsitfoundsub
    ON incsitfound.incsitfoundsub_id = incsitfoundsub.incsitfoundsub_id
  LEFT JOIN incsitfoundclass
    ON incsitfoundsub.incsitfoundclass_id = incsitfoundclass.incsitfoundclass_id 
 WHERE incsitfoundclass.incsitfoundclass_id IN (1,2)
 GROUP BY s.fma, s.geom
 ORDER BY incidents_per_sqmi DESC;



In [20]:
df5

,fma,fma_area_mi,num_incidents,incidents_per_sqmi
0,1,1.024407,837,817.057805
1,21,0.952173,561,589.178517
2,3,2.444352,934,382.105324
3,4,2.367524,840,354.801051
4,13,3.335275,973,291.730043
5,7,5.704560,1191,208.780344
6,11,5.306509,1021,192.405211
7,9,3.451640,597,172.961243
8,19,4.188032,669,159.740914
9,23,2.002927,300,149.780824


In [21]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 4 columns):
fma                   31 non-null int64
fma_area_mi           31 non-null float64
num_incidents         31 non-null int64
incidents_per_sqmi    31 non-null float64
dtypes: float64(2), int64(2)
memory usage: 1.0 KB


In [22]:
# inner join df2, df3 and df5 together on 'fma'

joined = df3.join(df5.set_index('fma'), on='fma', how = 'inner',lsuffix='_med', rsuffix='_fire')
joined['med_fire_ratio'] = joined['num_incidents_med']/joined['num_incidents_fire'] #add ratio of medical:fire incidents
joined = joined.join(df2[['fma','num_incidents']].set_index('fma'), on='fma')


In [23]:
joined['weekly_total_incs'] = joined['num_incidents']/84
joined['weekly_med_incs'] = joined['num_incidents_med']/84
joined['weekly_fire_incs'] = joined['num_incidents_fire']/84
joined.sort_values(by='fma',ascending=True)

,fma,fma_area_mi_med,num_incidents_med,incidents_per_sqmi_med,fma_area_mi_fire,num_incidents_fire,incidents_per_sqmi_fire,med_fire_ratio,num_incidents,weekly_total_incs,weekly_med_incs,weekly_fire_incs
0,1,1.024407,33984,33174.303996,1.024407,837,817.057805,40.602151,43139,513.559524,404.571429,9.964286
24,2,14.399547,9068,629.742048,14.399547,434,30.139838,20.894009,12804,152.428571,107.952381,5.166667
3,3,2.444352,19821,8108.896807,2.444352,934,382.105324,21.221627,30723,365.750000,235.964286,11.119048
2,4,2.367524,20167,8518.181895,2.367524,840,354.801051,24.008333,29829,355.107143,240.083333,10.000000
23,5,4.047208,5300,1309.544835,4.047208,226,55.840968,23.451327,7834,93.261905,63.095238,2.690476
29,6,3.297340,1027,311.463139,3.297340,159,48.220681,6.459119,2477,29.488095,12.226190,1.892857
5,7,5.704560,32937,5773.802010,5.704560,1191,208.780344,27.654912,42676,508.047619,392.107143,14.178571
14,8,3.673442,9199,2504.190743,3.673442,509,138.562136,18.072692,12831,152.750000,109.511905,6.059524
9,9,3.451640,11703,3390.561845,3.451640,597,172.961243,19.603015,16371,194.892857,139.321429,7.107143
27,10,4.255011,2108,495.415877,4.255011,137,32.197332,15.386861,3770,44.880952,25.095238,1.630952


In [24]:
# export csv to data folder for import to db
df_import = joined.sort_values(by='med_fire_ratio',ascending=False)
df_import.pop('fma_area_mi_fire') # delete the redundant data column
#df_import 
df_import.to_csv("responseTimeMetricsData/incidents_persqmi_fma.csv", index=False)